# FileToLink Bot - One-Click Deploy to Heroku

This notebook deploys your FileToLink Telegram Bot to Heroku **without installing anything** on your computer.

## What You Need Before Starting

Make sure you have these **4 things** ready:

| What | Where to get it | Example |
|------|----------------|--------|
| **Heroku API Key** | heroku.com > Account Settings > scroll to API Key > click Reveal | `a1b2c3d4-e5f6-...` |
| **Telegram Bot Token** | Message @BotFather on Telegram > /newbot | `7123456789:AAHx2...` |
| **Telegram API ID** | my.telegram.org > API development tools | `12345678` |
| **Telegram API Hash** | my.telegram.org > API development tools | `a1b2c3d4e5f6...` |

## How to Use This Notebook

1. Fill in your details in **Step 1** below
2. Click **Runtime** in the top menu, then **Run all**
3. Wait about 5 minutes - the notebook does everything for you
4. When done, you'll see your bot's URL at the bottom

---

## Step 1: Fill In Your Details

Click on the cell below, fill in each field, then move to Step 2.

**Tips:**
- `HEROKU_APP_NAME` = a unique name for your app (lowercase letters, numbers, and hyphens only). Example: `my-file-bot-2024`
- `HEROKU_EMAIL` = the email you used to sign up on Heroku
- `HEROKU_API_KEY` = Find it at: heroku.com > click your profile picture (top right) > Account Settings > scroll down to "API Key" > click "Reveal"
- `BOT_TOKEN` = The token BotFather gave you when you created your bot
- `API_ID` = The number from my.telegram.org
- `API_HASH` = The text string from my.telegram.org

In [ ]:
#@title Fill in your details here (click the arrow on the left to run this cell)

#@markdown ### Heroku Account Details
HEROKU_EMAIL = "" #@param {type:"string"}
HEROKU_API_KEY = "" #@param {type:"string"}
HEROKU_APP_NAME = "" #@param {type:"string"}

#@markdown ---
#@markdown ### Telegram Bot Details
BOT_TOKEN = "" #@param {type:"string"}
API_ID = "" #@param {type:"string"}
API_HASH = "" #@param {type:"string"}

#@markdown ---
#@markdown ### GitHub Repository (don't change unless you forked the repo)
GITHUB_REPO = "https://github.com/kakarotoncloud/Filetolink" #@param {type:"string"}

# Check that everything is filled in
errors = []
if not HEROKU_EMAIL: errors.append("HEROKU_EMAIL is empty")
if not HEROKU_API_KEY: errors.append("HEROKU_API_KEY is empty")
if not HEROKU_APP_NAME: errors.append("HEROKU_APP_NAME is empty")
if not BOT_TOKEN: errors.append("BOT_TOKEN is empty")
if not API_ID: errors.append("API_ID is empty")
if not API_HASH: errors.append("API_HASH is empty")

if errors:
    print("ERROR: Please fill in these fields:")
    for e in errors:
        print(f"  - {e}")
    raise ValueError("Please fill in all required fields above before continuing.")
else:
    HEROKU_APP_NAME = HEROKU_APP_NAME.lower().strip()
    print("All details filled in correctly!")
    print(f"  App name: {HEROKU_APP_NAME}")
    print(f"  Email: {HEROKU_EMAIL}")
    print(f"  Your app will be at: https://{HEROKU_APP_NAME}.herokuapp.com")

---
## Step 2: Install Heroku (automatic)

This installs the Heroku command-line tool in Colab. Just run it and wait.

In [ ]:
#@title Install Heroku CLI (click the arrow to run)
print("Installing Heroku CLI... (takes about 30 seconds)")
!curl -s https://cli-assets.heroku.com/install.sh | sh > /dev/null 2>&1
result = !heroku --version
print(f"Installed: {result[0]}")
print("Done!")

---
## Step 3: Login to Heroku (automatic)

This logs into your Heroku account using your API key. No browser needed.

In [ ]:
#@title Login to Heroku (click the arrow to run)
import os
os.environ["HEROKU_API_KEY"] = HEROKU_API_KEY

with open(os.path.expanduser("~/.netrc"), "w") as f:
    f.write(f"machine api.heroku.com\n")
    f.write(f"  login {HEROKU_EMAIL}\n")
    f.write(f"  password {HEROKU_API_KEY}\n")
    f.write(f"machine git.heroku.com\n")
    f.write(f"  login {HEROKU_EMAIL}\n")
    f.write(f"  password {HEROKU_API_KEY}\n")
os.chmod(os.path.expanduser("~/.netrc"), 0o600)

result = !heroku auth:whoami 2>&1
if "Error" in result[0] or "not" in result[0].lower():
    print(f"LOGIN FAILED: {result[0]}")
    print("Please check your HEROKU_EMAIL and HEROKU_API_KEY in Step 1.")
    print("")
    print("How to find your Heroku API Key:")
    print("1. Go to heroku.com")
    print("2. Click your profile picture (top right)")
    print("3. Click 'Account Settings'")
    print("4. Scroll down to 'API Key'")
    print("5. Click 'Reveal' and copy it")
else:
    print(f"Logged in as: {result[0]}")
    print("Login successful!")

---
## Step 4: Download the Bot Code (automatic)

This downloads the bot code from GitHub.

In [ ]:
#@title Download bot code from GitHub (click the arrow to run)
import os, shutil

if os.path.exists("/content/Filetolink"):
    shutil.rmtree("/content/Filetolink")

print(f"Downloading from {GITHUB_REPO}...")
!git clone {GITHUB_REPO} /content/Filetolink 2>&1 | tail -3

os.chdir("/content/Filetolink")

files = os.listdir(".")
print(f"\nDownloaded {len(files)} items successfully!")
print("Bot code is ready.")

---
## Step 5: Create Your App on Heroku (automatic)

This creates your Heroku app and adds a PostgreSQL database to it.

**Note:** The PostgreSQL database costs ~$5/month. You need a payment method on your Heroku account.

In [ ]:
#@title Create Heroku app + database (click the arrow to run)
import subprocess

# Check if app already exists
result = subprocess.run(["heroku", "apps:info", HEROKU_APP_NAME], capture_output=True, text=True)

if result.returncode != 0:
    print(f"Creating new app: {HEROKU_APP_NAME}...")
    !heroku create {HEROKU_APP_NAME} 2>&1
else:
    print(f"App '{HEROKU_APP_NAME}' already exists. Using it.")
    !heroku git:remote -a {HEROKU_APP_NAME} 2>&1

print("")

# Check if PostgreSQL is already added
result = subprocess.run(["heroku", "addons", "--app", HEROKU_APP_NAME], capture_output=True, text=True)
if "postgresql" not in result.stdout.lower():
    print("Adding PostgreSQL database... (this may take a minute)")
    !heroku addons:create heroku-postgresql:essential-0 --app {HEROKU_APP_NAME} 2>&1
else:
    print("PostgreSQL database already exists.")

print("")
print(f"Your app URL: https://{HEROKU_APP_NAME}.herokuapp.com")
print("App and database created!")

---
## Step 6: Set Your Bot's Keys (automatic)

This sends your Telegram keys to Heroku so the bot can use them.

In [ ]:
#@title Set environment variables on Heroku (click the arrow to run)
base_url = f"https://{HEROKU_APP_NAME}.herokuapp.com"

print("Setting your bot's configuration...")
!heroku config:set \
  BOT_TOKEN="{BOT_TOKEN}" \
  API_ID="{API_ID}" \
  API_HASH="{API_HASH}" \
  BASE_URL="{base_url}" \
  NODE_ENV="production" \
  --app {HEROKU_APP_NAME} 2>&1

print("")
print("Configuration set:")
print(f"  Bot Token: {'*' * 10}...{BOT_TOKEN[-5:]}")
print(f"  API ID: {API_ID}")
print(f"  API Hash: {'*' * 10}...{API_HASH[-5:]}")
print(f"  App URL: {base_url}")
print("")
print("All keys are set!")

---
## Step 7: Deploy! (automatic)

This is the main step - it uploads your bot to Heroku and starts it.

**This takes 2-3 minutes.** You'll see a lot of text scrolling - that's normal. Wait until you see "Done!" at the bottom.

In [ ]:
#@title Deploy to Heroku (click the arrow to run - takes 2-3 minutes)
import os
os.chdir("/content/Filetolink")

# Configure git
!git config user.email "{HEROKU_EMAIL}"
!git config user.name "FileToLink Deploy"

# Add Heroku remote
!heroku git:remote -a {HEROKU_APP_NAME} 2>&1

# Set Node.js buildpack
!heroku buildpacks:set heroku/nodejs --app {HEROKU_APP_NAME} 2>&1

print("")
print("Uploading and building your bot on Heroku...")
print("(This takes 2-3 minutes, please wait...)")
print("")

# Push to Heroku
!git push heroku main --force 2>&1

# Scale web dyno
!heroku ps:scale web=1 --app {HEROKU_APP_NAME} 2>&1

print("")
print("="*50)
print("DEPLOYMENT COMPLETE!")
print("="*50)

---
## Step 8: Check If Everything is Working

Let's verify your bot is running correctly.

In [ ]:
#@title Check bot status (click the arrow to run)
import time, urllib.request, json

print("Waiting 20 seconds for the bot to start up...")
time.sleep(20)

base_url = f"https://{HEROKU_APP_NAME}.herokuapp.com"

# Check if the website is up
print("\nChecking website...")
try:
    req = urllib.request.urlopen(f"{base_url}/api/bot-info", timeout=15)
    data = json.loads(req.read())
    print(f"  Website is UP")
    print(f"  Bot name: {data.get('name', 'Unknown')}")
    print(f"  Bot username: @{data.get('username', 'Unknown')}")
except Exception as e:
    print(f"  Website not ready yet - this is normal, wait 1-2 minutes and try again")
    print(f"  Error: {e}")

# Check stats
print("\nChecking stats...")
try:
    req = urllib.request.urlopen(f"{base_url}/api/stats", timeout=15)
    data = json.loads(req.read())
    print(f"  Total files: {data.get('totalFiles', 0)}")
    print(f"  Total downloads: {data.get('totalDownloads', 0)}")
except Exception as e:
    print(f"  Stats not available yet")

# Show recent logs
print("\n" + "="*50)
print("Recent logs from your bot:")
print("="*50)
!heroku logs --num 20 --app {HEROKU_APP_NAME} 2>&1

print("\n" + "="*50)
print("YOUR BOT IS READY!")
print("="*50)
print(f"")
print(f"Website: {base_url}")
print(f"")
print(f"What to do now:")
print(f"1. Open Telegram on your phone")
print(f"2. Search for your bot's username")
print(f"3. Send it any file (photo, video, document)")
print(f"4. The bot will reply with a download link")
print(f"5. Open the link in any browser to download or stream!")
print(f"")
print(f"Your bot runs 24/7 - you can close this notebook.")

---
---
## Management Tools

Use these cells anytime you need to manage your bot. You don't need to run these now - come back later when needed.

---

In [ ]:
#@title View Live Logs (see what your bot is doing right now)
!heroku logs --num 50 --app {HEROKU_APP_NAME}

In [ ]:
#@title Restart Your Bot
!heroku restart --app {HEROKU_APP_NAME}
print("Bot restarted! It will be back online in about 30 seconds.")

In [ ]:
#@title Update Bot (get latest version from GitHub and redeploy)
import os
os.chdir("/content/Filetolink")
!git pull origin main 2>&1
!git push heroku main --force 2>&1
print("")
print("Bot updated and redeployed!")

In [ ]:
#@title Check Bot Status
!heroku ps --app {HEROKU_APP_NAME}
print("")
!heroku config --app {HEROKU_APP_NAME}

In [ ]:
#@title DELETE Your App (WARNING: This cannot be undone!)
# To delete your app, uncomment the line below (remove the # at the start) and run this cell
# !heroku apps:destroy {HEROKU_APP_NAME} --confirm {HEROKU_APP_NAME}
print("To delete your app, uncomment the line above (remove the #) and run this cell again.")
print("WARNING: This will permanently delete your app and all its data!")